Carga del archivo

In [1]:
from google.colab import files
uploaded = files.upload()
#rename to data.xlsx
import os

try:
    uploaded_filename = list(uploaded.keys())[0]
    os.rename(uploaded_filename, "data.xlsx")
except IndexError:
    print("No file uploaded.")
except FileNotFoundError:
    print(f"Error: File '{uploaded_filename}' not found.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")



Saving data.xlsx to data.xlsx


# **Instalación de Liberías**

**Streamlit:** biblioteca para construir aplicaciones web interactivas en Python, muy usada para dashboards de datos.

**plotly** → biblioteca para crear gráficos avanzados e interactivos, como los velocímetros (gauge) que usas en tu código.

**pandas** → biblioteca para manipulación y análisis de datos en forma de tablas (DataFrames).

**streamlit_elements** → extensión de Streamlit que permite agregar componentes adicionales, como diseños avanzados, menús o tablas interactivas.

In [2]:
!pip install streamlit plotly pandas streamlit_elements

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.0 MB/s eta 0:00:00


 Instalar local tunnels

In [3]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏
added 22 packages in 3s
⠏
⠏3 packages are looking for funding
⠏  run `npm fund` for details
⠏

Funciones para el manejo de datos, definir las condiciones de los semáforos, Calcular los KPIs

In [4]:
%%writefile utils.py
from typing import Tuple, List, Dict, Any, Optional
import pandas as pd
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
from plotly.graph_objects import Figure

def prepareDataTable(plot_data, is_higher_better):
    """Función que cuando es invocada, Prepara los datos para mostrarlos en una tabla con semáforos.

    Parámetros:
    -----------
    plot_data : pd.DataFrame
        Tabla con columnas de KPI, meta y aceptación.
    is_higher_better : bool
        Indica si valores más altos son mejores.

    Devuelve:
    --------
    display_data.DataFrame
        Tabla formateada para mostrar en el dashboard.
    """
    # Hacer copia de los datos para no alterar los originales
    display_data = plot_data.copy()

    # Si en los datos hay columnas Fecha, darle formato estándar (año/mes)
    if 'Fecha' in display_data.columns:
        display_data['Fecha'] = display_data['Fecha'].dt.strftime('%Y/%m')

    # Aplica la función getSemaphore para calcular el estado del semáforo (rojo, amarillo, verde)
    display_data['Estado'] = display_data.apply(
        lambda row: getSemaphore(row, is_higher_better), #Como is_higher_better es bool, cuando es false es porque bajo es mejor
        axis=1
    )

    # La Función devuelve los datos preparados
    return display_data


def loadData(filePath: str = "data.xlsx") -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    ### Función que Carga los datos desde el archivo Excel y devuelve varias tablas usando a pd que es la instancia de la librería Pandas
    ### Tiene su respectivo control de errores.

    try:
        # En este primer bloque se leen hojas específicas del archivo Excel (cada hoja queda en una variable)
        mediciones = pd.read_excel(filePath, sheet_name=0)
        #Toma la hoja de excel llamada Errores_Trim que contiene la cantidad de errores de las aplicaciones por trimestre
        errores = pd.read_excel(filePath, sheet_name=1)
        tiempoDeRespuesta = pd.read_excel(filePath, sheet_name=2)
        disponibilidad = pd.read_excel(filePath, sheet_name=3)
        tiquetesConErrores = pd.read_excel(filePath, sheet_name=4)
    except FileNotFoundError:
        raise FileNotFoundError("El archivo data.xlsx no se encuentra en la ruta especificada.")
    except ValueError:
        raise ValueError("El archivo data.xlsx no contiene las hojas necesarias.")
    except Exception as e:
        raise Exception(f"Error al cargar el archivo: {e}")

    try:
        # En este segundo bloque se convierten las columnas de fecha para que el sistema las entienda como fechas y se puedan operar
        mediciones['Fecha_Inicio_Mes'] = pd.to_datetime(mediciones['Fecha_Inicio_Mes'])
        mediciones['Fecha_Fin_Mes'] = pd.to_datetime(mediciones['Fecha_Fin_Mes'])
        tiempoDeRespuesta['Fecha_Inicio_Mes'] = pd.to_datetime(tiempoDeRespuesta['Fecha_Inicio_Mes'])
        tiempoDeRespuesta['Fecha_Fin_Mes'] = pd.to_datetime(tiempoDeRespuesta['Fecha_Fin_Mes'])
        disponibilidad['Fecha_Inicio_Mes'] = pd.to_datetime(disponibilidad['Fecha_Inicio_Mes'])
        disponibilidad['Fecha_Fin_Mes'] = pd.to_datetime(disponibilidad['Fecha_Fin_Mes'])
    except KeyError as e:
        raise KeyError(f"Error al convertir columnas a datetime: {e}")
    except Exception as e:
        raise Exception(f"Error inesperado: {e}")

    #se retornan las variables que contienen cada una de las hojas de datos ya con los formatos de fecha tratados
    return mediciones, tiempoDeRespuesta, disponibilidad, tiquetesConErrores, errores


def preprocessErrores(errores: pd.DataFrame) -> pd.DataFrame:
    #Procesa los datos que se cargaron de la hoja Errores_Trim en la variable errores, para agregar columna de fecha si es necesario.

    if 'Fecha' not in errores.columns and 'Año' in errores.columns and 'Trimestre' in errores.columns:
        # Asociar trimestre a primer mes correspondiente (ej. Trim.1 → enero)
        trimestre_a_mes = {
            'Trim.1': 1,
            'Trim.2': 4,
            'Trim.3': 7,
            'Trim.4': 10
        }

        # Crear una columna de fecha combinando año y mes
        errores['Fecha'] = errores.apply(
            lambda row: pd.Timestamp(year=row['Año'], month=trimestre_a_mes[row['Trimestre']], day=1),
            axis=1
        )

        # Ordenar por fecha para facilitar los análisis
        errores = errores.sort_values(['Aplicación', 'Criticidad', 'Fecha'])

    #Se retornan los datos de errores, con la transformación para ser procesados posteriormente
    return errores


def setupFilters(mediciones):
    #En esta función se configuran los filtros en la barra lateral del dashboard para devolver los datos filtrados.
    #Los filtros son: por aplicación, por año, por mes

    # Obtiene de los datos de mediciones(que corresponden a la hoja Mediciones_APP) lista de aplicaciones para el filtro
    services = mediciones['Aplicación'].unique().tolist()

    # También de la columna Fecha_Inicio_Mes de mediciones, se obtiene lista de años y meses disponibles para
    years = sorted(mediciones['Fecha_Inicio_Mes'].dt.year.unique().tolist())
    months = list(range(1, 13))

    # Se Crean las secciones de filtros que serán desplegados en la barra lateral del tablero de streamlit
    st.sidebar.header("Dashboard de KPIs")
    st.sidebar.subheader("Filtros")
    selected_service = st.sidebar.selectbox("Seleccionar servicio", services)
    selected_kpi = st.sidebar.selectbox("Seleccionar KPI", {"Tiempo Respuesta": "Tiempo Respuesta", "Disponibilidad": "Horas_Disp_reales_mes", "Tiquetes con Errores": "Tiquetes con Errores"})

    selected_year = st.sidebar.selectbox("Seleccionar año", ["Todos"] + years)
    selected_month = st.sidebar.selectbox("Seleccionar mes", ["Todos"] + [f"{m:02d}" for m in months])


    # Filtrar datos según las selecciones que hace el usuario en el tablero
    service_data = mediciones[mediciones['Aplicación'] == selected_service]

    if selected_year != "Todos":
        service_data = service_data[service_data['Fecha_Inicio_Mes'].dt.year == selected_year]

    if selected_month != "Todos":
        selected_month_int = int(selected_month)
        service_data = service_data[service_data['Fecha_Inicio_Mes'].dt.month == selected_month_int]

    service_data = service_data.sort_values('Fecha_Inicio_Mes')

    #Retorna las opciones filtradas por los usuarios
    return selected_service, selected_kpi, service_data


def calculateKpiValues(service_data, kpi_name, tiempoDeRespuesta, disponibilidad, tiquetesConErrores):
    # Esta función va a ser llamada para calcular los valores de todos los KPIs de acuerdo con la fórmula de cada indicador.
    # La función recibe el nombre del KPI

    #Se calcula el indicador de tiempo de respuesta
    if kpi_name == "Tiempo Respuesta":
        # Extraer valores para calcular el indicador. Tiempo de Respuesta
        kpi_values = service_data["Tiempo Respuesta"].tolist()

        # Aquí se obtiene la meta y nivel aceptable de otro conjunto de datos (que tiene cargados los valores de la hoja Metas_ToR)
        meta_values = [tiempoDeRespuesta.loc[tiempoDeRespuesta['Fecha_Inicio_Mes'] == date, 'Meta_ToR'].values[0]
                      if date in tiempoDeRespuesta['Fecha_Inicio_Mes'].values else None
                      for date in service_data['Fecha_Inicio_Mes']]
        nivel_aceptable = [tiempoDeRespuesta.loc[tiempoDeRespuesta['Fecha_Inicio_Mes'] == date, 'Nivel_Aceptable'].values[0]
                           if date in tiempoDeRespuesta['Fecha_Inicio_Mes'].values else None
                           for date in service_data['Fecha_Inicio_Mes']]
        y_axis_label = "Tiempo de Respuesta"

    #Se calcula el indicador Disponibilidad
    elif kpi_name == "Disponibilidad":
        # Extraer valores para calcular el indicador Disponibilidad. En el excel están en la hoja Mediciones_App
        # Se aplica la fórmula, multiplicando por 100 para expresar en términos porcentuales
        kpi_values = ((service_data["Horas_Disp_reales_mes"] /
                      (service_data["Horas_Disp_Mes"] - service_data["Horas_Indisp_Progr_mes"])) * 100).tolist()
        #Los valores para las metas de disponibilidad son los que están en el excel en la hoja Metas_disp
        meta_values = [disponibilidad.loc[disponibilidad['Fecha_Inicio_Mes'] == date, 'Meta_disp'].values[0] * 100
                      if date in disponibilidad['Fecha_Inicio_Mes'].values else None
                      for date in service_data['Fecha_Inicio_Mes']]
        nivel_aceptable = [disponibilidad.loc[disponibilidad['Fecha_Inicio_Mes'] == date, 'Nivel_Aceptable'].values[0] * 100
                           if date in disponibilidad['Fecha_Inicio_Mes'].values else None
                           for date in service_data['Fecha_Inicio_Mes']]
        y_axis_label = "Disponibilidad (%)"

    #Se calcula el indicador Tiquetes con Errores TCE
    elif kpi_name == "Tiquetes con Errores":
         # Se obtiene el número de tickets con error
        kpi_values = service_data["Tiquetes con Errores"].tolist()

        current_year = service_data['Fecha_Inicio_Mes'].dt.year.iloc[0] if not service_data.empty else 2020
        criticality = "Alta"
        #se obtienen las metas desde la variable que equivale a la hoja de excel "Metas_TCE"
        meta_values = [tiquetesConErrores.loc[(tiquetesConErrores['Criticidad'] == criticality) &
                                              (tiquetesConErrores['Año'] == current_year), 'Meta'].values[0]
                       if not tiquetesConErrores[(tiquetesConErrores['Criticidad'] == criticality) &
                                                (tiquetesConErrores['Año'] == current_year)].empty else None]
        nivel_aceptable = [meta_values[0]] if meta_values else [None]
        y_axis_label = "Numero de Tiquetes con Errores"

    #Retorna cada indicador según sea llamado con toda la información para ser graficado, los valores, las metas, los niveles de referencia
    return kpi_values, meta_values, nivel_aceptable, y_axis_label

def calculateTCEWithErroresTable(service_data, selected_kpi, errores, selected_service, tiquetesConErrores=None, criticality=None):

    #Calcula el KPI de tickets con error usando la tabla de errores.

    if selected_kpi == "Tiquetes con Errores":
        # Si no se proporcionan metas, mostrar advertencia y usar valores predeterminados
        if tiquetesConErrores is None:
            st.warning("Datos de metas para tiquetes no proporcionados. Se usarán valores predeterminados.")

        # Filtrar los datos de errores solamente para el servicio seleccionado
        service_errors = errores[errores['Aplicación'] == selected_service].sort_values('Fecha')

        # Si no hay datos de errores para ese servicio, devolver vacío
        if service_errors.empty:
            return [], [], [], "Porcentaje de Tiquetes Resueltos"

        # Obtiene datos filtrando por el rango de fechas disponible en el conjunto principal
        if not service_data.empty:
            # Obtiene las fechas min y max del service_data
            min_date = service_data['Fecha_Inicio_Mes'].min()
            max_date = service_data['Fecha_Inicio_Mes'].max()

            # Si hay columna de fecha, filtrar por fechas; si solo hay año, filtrar por año
            if 'Fecha' in service_errors.columns:
                service_errors = service_errors[
                    (service_errors['Fecha'] >= min_date) &
                    (service_errors['Fecha'] <= max_date)
                ]
            elif 'Año' in service_errors.columns:
                # If we only have year, filter by year
                min_year = min_date.year
                max_year = max_date.year
                service_errors = service_errors[
                    (service_errors['Año'] >= min_year) &
                    (service_errors['Año'] <= max_year)
                ]

        # Si después del filtrado no hay datos, devolver listas vacías
        if service_errors.empty:
            return [], [], [], "Porcentaje de Tiquetes Resueltos"

        # Obtener la criticidad disponible (si no se especifica, usar la primera que aparece)
        criticalities = service_errors['Criticidad'].unique()

        # Si no hay criticidades disponibles, devolver listas vacías
        if len(criticalities) == 0:
            return [], [], [], "Porcentaje de Tiquetes Resueltos"

        # Si se proporciona una criticidad específica, usarla
        if criticality is not None:
            if criticality in criticalities:
                selected_criticality = criticality
            else:
                return [], [], [], "Porcentaje de Tiquetes Resueltos"

        # Permitir al usuario seleccionar la criticidad si hay más de una y no se proporciona una específica
        elif len(criticalities) > 1 and criticality is None:
            selected_criticality = st.sidebar.selectbox("Seleccionar Criticidad", criticalities, key=f"crit_select_{selected_service}")
        else:
            selected_criticality = criticalities[0]

        # Filtrar los datos de errores por criticidad seleccionada
        service_errors_by_crit = service_errors[service_errors['Criticidad'] == selected_criticality]

        # Si no hay datos tras filtrar por criticidad, devolver vacío
        if service_errors_by_crit.empty:
            return [], [], [], "Porcentaje de Tiquetes Resueltos"

        # Calcular el KPI como porcentaje de tiquetes resueltos
        kpi_values = []
        for i in range(len(service_errors_by_crit)):
            total_errors = service_errors_by_crit['Cant_Tiq_Error_Trim'].iloc[i]
            closed_errors = service_errors_by_crit['Cant_Tiq_Error_Cerr'].iloc[i]

            # Si no hubo errores, se considera 100% resuelto; si hubo, se calcula el % resuelto
            if total_errors == 0:
                pct_resolved = 100  # Si no hay errores, 100% resueltos
            else:
                pct_resolved = (closed_errors / total_errors) * 100

            kpi_values.append(pct_resolved)

        # Para el servicio seleccionado, obtener la meta de tiquetes según criticidad y año
        years = service_errors_by_crit['Año'].unique()
        meta_values = []
        nivel_aceptable = []

        for year in years:
            if tiquetesConErrores is not None:
                # Buscar en el dataframe tiquetesConErrores para obtener los valores de meta
                meta_row = tiquetesConErrores[
                    (tiquetesConErrores['Criticidad'] == selected_criticality) &
                    (tiquetesConErrores['Año'] == year)
                ]

                if not meta_row.empty:
                    # FIXED: Convert Meta to percentage (multiply by 100) if it's a ratio
                    meta = meta_row['Meta'].values[0]
                    if meta <= 1:  # If meta is expressed as a ratio (0-1)
                        meta = meta * 100  # Convert to percentage (0-100)
                    meta_values.extend([meta] * 4)  # Un valor para cada trimestre

                    # Check if 'Nivel_Aceptable' column exists in tiquetesConErrores
                    if 'Nivel_Aceptable' in meta_row.columns:
                        accept = meta_row['Nivel_Aceptable'].values[0]
                        if accept <= 1:  # If acceptance is expressed as a ratio (0-1)
                            accept = accept * 100  # Convert to percentage (0-100)
                        nivel_aceptable.extend([accept] * 4)
                    else:
                        # Use default of 80% of meta
                        nivel_aceptable.extend([meta * 0.8] * 4)
                else:
                    # Valores predeterminados si no se encuentra - already in percentage (0-100)
                    meta_values.extend([80] * 4)  # 80% resueltos como meta
                    nivel_aceptable.extend([64] * 4)  # 64% como aceptable
            else:
                # Si no se proporciona tiquetesConErrores, usar valores predeterminados - already in percentage (0-100)
                meta_values.extend([80] * 4)  # 80% resueltos como meta
                nivel_aceptable.extend([64] * 4)  # 64% como aceptable

        # Asegurar que las listas tengan el mismo tamaño que los valores calculados
        meta_values = meta_values[:len(kpi_values)]
        nivel_aceptable = nivel_aceptable[:len(kpi_values)]

        # Retornar la información de los tiquetes con errores y resueltos
        return kpi_values, meta_values, nivel_aceptable, f"Porcentaje de Tiquetes Resueltos ({selected_criticality})"

        # Si el KPI no es de tiquetes con errores, devolver vacío
        return [], [], [], "Unknown KPI"


def calculateAllKpiValues(service_data, errores, tiempoDeRespuesta, disponibilidad, tiquetesConErrores, selected_service):
    """Calculate values for all KPIs at once."""
    # Para Tiempo de Respuesta
    tor_service_data = service_data.copy()
    ToR = calculateKpiValues(tor_service_data, "Tiempo Respuesta", tiempoDeRespuesta, disponibilidad, tiquetesConErrores)
    kpi_ToR = ToR[0]
    meta_ToR = ToR[1]
    acceptance_ToR = ToR[2]

    # Para Disponibilidad
    disp_service_data = service_data.copy()
    Disp = calculateKpiValues(disp_service_data, "Disponibilidad", tiempoDeRespuesta, disponibilidad, tiquetesConErrores)
    kpi_Disp = Disp[0]
    meta_Disp = Disp[1]
    acceptance_Disp = Disp[2]

    # Para Tiquetes con Errores - pasando tiquetesConErrores como parámetro
    Tiq = calculateTCEWithErroresTable(service_data, "Tiquetes con Errores", errores, selected_service, tiquetesConErrores)
    kpi_Tiquetes = Tiq[0]
    meta_Tiquetes = Tiq[1]
    acceptance_Tiquetes = Tiq[2]

    return {
        'ToR': {
            'kpi': kpi_ToR,
            'meta': meta_ToR,
            'acceptance': acceptance_ToR
        },
        'Disp': {
            'kpi': kpi_Disp,
            'meta': meta_Disp,
            'acceptance': acceptance_Disp
        },
        'Tiq': {
            'kpi': kpi_Tiquetes,
            'meta': meta_Tiquetes,
            'acceptance': acceptance_Tiquetes
        }
    }

###########################################################################
# HASTA ACÁ SE HACE TODO EL PROCESAMIENTO DE LOS DATOS Y EL CÁLCULO DE LOS#
# INDICADORES. A PARTIR DE ESTE PUNTO SE USA EL RESULTADO ANTERIOR PARA   #
# CONSTRUIR LOS OBJETOS GRÁFICOS QUE LUEGO SERÁN COLOCADOS EN EL TABLERO  #
###########################################################################

def createGauge(value, meta, acceptance, title, is_higher_better):
    # Crea un gráfico tipo velocímetro (gauge) para visualizar indicadores clave (KPIs).
    if is_higher_better:
        # Para indicadores donde un valor más alto es mejor (ej. disponibilidad del sistema)
        steps = [
            {'range': [0, acceptance], 'color': 'red'},
            {'range': [acceptance, meta], 'color': 'yellow'},
            {'range': [meta, max(110, 1.1 * meta)], 'color': 'green'}
        ]
        # Asegurar que la escala llegue al menos a 110% para que se vea bien, se usa al pintar el gráfico
        max_val = max(110, 1.1 * meta)
    else:
        # Para indicadores donde un valor más bajo es mejor (ej. tiempo de respuesta, número de errores)
        steps = [
            {'range': [0, meta], 'color': 'green'},
            {'range': [meta, acceptance], 'color': 'yellow'},
            {'range': [acceptance, 2 * acceptance], 'color': 'red'}
        ]
        max_val = 2 * acceptance # Escala máxima adaptada a estos casos, se usa al pintar el gráfico

    # Crear el gráfico usando la biblioteca Plotly
    fig = go.Figure(go.Indicator(
        mode="gauge+number", # Mostrar tanto el dial como el número
        value=value, # Valor actual del KPI
        domain={'x': [0, 1], 'y': [0, 1]}, # Ocupa todo el espacio disponible
        title={'text': title}, # Título del gráfico (ej. "Disponibilidad")
        gauge={
            'axis': {'range': [0, max_val]}, # Rango del dial (0 al máximo calculado)
            'bar': {'color': "#1f77b4"}, # Color de la aguja
            'steps': steps,  # Definir las zonas de color
            'threshold': {
                'line': {'color': "black", 'width': 3}, # Línea que marca la meta
                'thickness': 0.75,
                'value': meta # Valor de la meta a marcar
            }
        }
    ))

    # Colocar el % cuando la métrica se calcula en términos procentuales
    if title == "Disponibilidad" or title == "% Tiquetes Resueltos":
        fig.update_traces(number={'suffix': '%'})

    # Ajustar el tamaño y los márgenes para que el gráfico se vea ordenado
    fig.update_layout(height=200, margin=dict(l=30, r=30, t=50, b=30))
    return fig


Writing utils.py


Pintamos los tacómetros en la UI

In [5]:
%%writefile app.py
import streamlit as st

#Se importan las funciones que se definieron en la celda anterior
from utils import (loadData, preprocessErrores, setupFilters, calculateAllKpiValues, createGauge)

# Se Configura la página del dashboard
st.set_page_config(page_title="Medidores de KPIs", layout="wide")

# Cargar datos de los archivos (se cargan mediciones y tablas auxiliares)
mediciones, tiempoDeRespuesta, disponibilidad, tiquetesConErrores, errores = loadData()
# Procesar los datos de errores para agregarles columna de fecha, si es necesario
errores = preprocessErrores(errores)

# Configurar filtros para que el usuario pueda elegir qué servicio ver
selected_service, _, service_data = setupFilters(mediciones)

# Calcular todos los KPIs (Tiempo de Respuesta, Disponibilidad, Tiquetes con errores)
kpis = calculateAllKpiValues(
    service_data, errores, tiempoDeRespuesta, disponibilidad, tiquetesConErrores, selected_service
)

# Crear la estructura visual del tablero (dashboard) "layout"
st.title(f"Medidores de KPIs para {selected_service}")
col1, col2, col3 = st.columns(3)

# Mostrar el velocímetro de Tiempo de Respuesta (si hay datos)
if kpis['ToR']['kpi']:
    with col1:
        st.subheader("Tiempo de Respuesta")
        gauge = createGauge(
            kpis['ToR']['kpi'][-1], kpis['ToR']['meta'][-1], kpis['ToR']['acceptance'][-1],
            "Tiempo de Respuesta", is_higher_better=False
        )
        st.plotly_chart(gauge, use_container_width=True)

# Mostrar el velocímetro de Disponibilidad (si hay datos)
if kpis['Disp']['kpi']:
    with col2:
        st.subheader("Disponibilidad")
        gauge = createGauge(
            kpis['Disp']['kpi'][-1], kpis['Disp']['meta'][-1], kpis['Disp']['acceptance'][-1],
            "Disponibilidad", is_higher_better=True
        )
        st.plotly_chart(gauge, use_container_width=True)

# Mostrar el velocímetro de Disponibilidad (si hay datos)
if kpis['Tiq']['kpi']:
    with col3:
        st.subheader("Tiquetes Resueltos")
        gauge = createGauge(
            kpis['Tiq']['kpi'][-1], kpis['Tiq']['meta'][-1], kpis['Tiq']['acceptance'][-1],
            "Tiquetes Resueltos", is_higher_better=True
        )
        st.plotly_chart(gauge, use_container_width=True)

Writing app.py


Obtener IP (Utilizar como contraseña de la url en el proximo paso)

In [6]:
!wget -q -O - ipv4.icanhazip.com

35.199.7.122


Exponer tunel

In [7]:
!streamlit run app.py & npx localtunnel --port 8501

⠙⠹⠸your url is: https://clean-dodos-unite.loca.lt



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.199.7.122:8501

  Stopping...
^C
